In [1]:
import os 
from collections import namedtuple as tup
import argparse


import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from sched import SchedT1Dataset
from sched_solver import Solver
from sched_heuristic import get_rm_solution
from sched_heuristic import liu_test
import sched_heuristic as heu
import pickle

In [17]:
Args = tup(
    "Argument",
    "seq_len num_procs num_sample period_range util")


NUM_SAMPLES = 10000
GRANUALITY = 20 

In [18]:

step = 8 / 20
util_range = np.arange(step, 8, step)

In [170]:
## data/#procs_#tasks
import sched
import importlib
importlib.reload(sched)
def gen_taskset(num_procs, seq_len, ddir="data"):
    try:
        os.mkdir(ddir)
    except FileExistsError:
        pass 
    base = "%s/%d-%d" % (ddir, num_procs, seq_len)
    try:
        os.mkdir(base)
    except FileExistsError:
        pass 

    
    step = 0.05

    for util in np.arange(step, num_procs, step):
        args = Args(seq_len=seq_len, 
            num_procs=num_procs, 
            num_sample=NUM_SAMPLES, 
            period_range=(10, 10000), 
            util=util)
        dataset = sched.SchedT1Dataset(args.num_procs, args.seq_len, args.num_sample, args.period_range, args.util, gg=True)
        with open(os.path.join(base, "%0.2f" % util), 'wb') as f:
            pickle.dump(dataset, f)


In [171]:
importlib.reload(sched)
base="TMP"
args = Args(seq_len=16, 
            num_procs=2, 
            num_sample=10000, 
            period_range=(2, 512), 
            util=0.3)
dataset = sched.SchedT1Dataset(args.num_procs, args.seq_len, args.num_sample, args.period_range, args.util, gg=True)
with open("TMP", 'wb') as f:
    pickle.dump(dataset, f)

int64
int64
int64


In [172]:
with open("TMP", "rb") as f:
    ret = pickle.load(f)

In [113]:
x = np.array([ret[0][1], ret[1][1]])

In [114]:
x

array([[[  56,    1,   24],
        [ 423,   17,   59],
        [ 117,    1,   21],
        [ 145,   12,   70],
        [  37,    3,   13],
        [ 290,    8,  183],
        [ 164,   16,   37],
        [ 148,    5,  107],
        [ 116,    6,   46],
        [  97,    2,   34]],

       [[  13,    1,   12],
        [4895,  878, 2302],
        [  16,    2,    2],
        [  17,    1,   15],
        [ 252,    4,   42],
        [  12,    1,    5],
        [ 573,   25,   94],
        [2569,  220,  465],
        [ 149,   26,   82],
        [ 749,   51,  583]]])

In [110]:
inputs = x

In [111]:
div = np.stack([
            (inputs[:, :, 1] / inputs[:, :, 0]), # Utilization
            (1.0 / inputs[:, :, 0] * 0.1)], axis=-1)

In [112]:
div

array([[[1.78571429e-02, 1.78571429e-03],
        [4.01891253e-02, 2.36406619e-04],
        [8.54700855e-03, 8.54700855e-04],
        [8.27586207e-02, 6.89655172e-04],
        [8.10810811e-02, 2.70270270e-03],
        [2.75862069e-02, 3.44827586e-04],
        [9.75609756e-02, 6.09756098e-04],
        [3.37837838e-02, 6.75675676e-04],
        [5.17241379e-02, 8.62068966e-04],
        [2.06185567e-02, 1.03092784e-03]],

       [[7.69230769e-02, 7.69230769e-03],
        [1.79366701e-01, 2.04290092e-05],
        [1.25000000e-01, 6.25000000e-03],
        [5.88235294e-02, 5.88235294e-03],
        [1.58730159e-02, 3.96825397e-04],
        [8.33333333e-02, 8.33333333e-03],
        [4.36300175e-02, 1.74520070e-04],
        [8.56364344e-02, 3.89256520e-05],
        [1.74496644e-01, 6.71140940e-04],
        [6.80907877e-02, 1.33511348e-04]]])

In [22]:
num_proc_list = [2]
num_tasks_list = [10]
for num_proc in num_proc_list:
    for num_tasks in num_tasks_list:
        if num_proc >= num_tasks:
            continue
        gen_taskset(num_proc, num_tasks, "gg")

KeyboardInterrupt: 

In [16]:
num_proc_list = [4, 8, 12, 16]
num_tasks_list = [4, 8, 16, 32, 48, 64, 80]
for num_proc in num_proc_list:
    for num_tasks in num_tasks_list:
        if num_proc >= num_tasks:
            continue
        gen_taskset(num_proc, num_tasks,"tr")

In [17]:
type(np.sum(np.array([1, 2, 3])))

numpy.int64